In [ ]:
#Feature importance
#catboost
import numpy as np
import pandas as pd
import os
import joblib
X_train = pd.read_csv('~/Library/CloudStorage/OneDrive-国立大学法人東海国立大学機構/Weekly_challenges/Data science and Analytics/Predict online behavior kaggle/Data/processed/x_train_online_gaming_behavior_dataset.csv')
logs_dir = "results/logs"
os.makedirs(logs_dir, exist_ok=True)
model_path = os.path.join(logs_dir, f"CatBoost_model.pkl")
model=joblib.load(model_path)

# Get feature importance
catboost_importance = model.get_feature_importance()

# Convert to DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': catboost_importance
}).sort_values(by="Importance", ascending=False)

# Display results
import ace_tools as tools
tools.display_dataframe_to_user(name="CatBoost Feature Importance", dataframe=feature_importance_df)
